In [1]:
import numpy as np
import cvxpy as cvx
import matplotlib.pyplot as plt

In [2]:
# US_3-MONTH_T-BILLS US_GOVN_LONG_BONDS SP_500 WILSHIRE_5000 :=
X = np.matrix([
    [1984, 1.103, 1.159, 1.061, 1.030],
    [1985, 1.080, 1.366, 1.316, 1.326],
    [1986, 1.063, 1.309, 1.186, 1.161],
    [1987, 1.061, 0.925, 1.052, 1.023],
    [1988, 1.071, 1.086, 1.165, 1.179],
    [1989, 1.087, 1.212, 1.316, 1.292],
    [1990, 1.080, 1.054, 0.968, 0.938],
    [1991, 1.057, 1.193, 1.304, 1.342],
    [1992, 1.036, 1.079, 1.076, 1.090],
    [1993, 1.031, 1.217, 1.100, 1.113],
    [1994, 1.045, 0.889, 1.012, 0.999]]);

n, d = np.shape(X)

L = 1 - X[:,1:n];
mu_est = -np.mean(L,0);
mu_est = np.array(mu_est)
mu_est

array([[0.06490909, 0.13536364, 0.14145455, 0.13572727]])

In [3]:
portfolio = np.array([0.25, 0.25, 0.25, 0.25])
wealthLoss = np.matmul(L, portfolio)
print(wealthLoss)

[[-0.08825 -0.272   -0.17975 -0.01525 -0.12525 -0.22675 -0.01    -0.224
  -0.07025 -0.11525  0.01375]]


In [4]:
wealthLoss.sort()
print("Sorted Portfolio Loss:")
print(wealthLoss)    # (a)  

Sorted Portfolio Loss:
[[-0.272   -0.22675 -0.224   -0.17975 -0.12525 -0.11525 -0.08825 -0.07025
  -0.01525 -0.01     0.01375]]


In [5]:
q = cvx.Variable(11)

constraints = [];

constraints += [sum(q) == 1]
constraints += [q <= 0.5]
constraints += [q >= 0]

obj = wealthLoss*q
objective = cvx.Maximize(obj);
prob = cvx.Problem(objective, constraints);

prob.solve()
if (prob.status == "optimal"):
    print('optimal value = ', prob.value)
    print('probability: ')
    print(q.value)    # (b) 

optimal value =  0.0018748653427732938
probability: 
[ 1.15059165e-07  1.19506229e-07  1.19078527e-07  1.14286855e-07
  1.16993948e-07  1.15197752e-07  1.18806206e-07  1.07177396e-07
 -9.27845482e-07  4.99999885e-01  5.00000116e-01]


In [7]:
x = cvx.Variable(4)   # portfolio weight
b = cvx.Variable(4, boolean = True)  #binary variables
beta = cvx.Variable(1)   # dual variable beta
# l = cvx.Variable(11)    # porfolio loss
l = L*x
z = cvx.Variable(11)

gamma = np.array([0.08, 0.06, 0.04, 0.02])

x_pos = cvx.Variable(4)
x_neg = cvx.Variable(4)

K = 10000
M = 1.25
constraints = [];
constraints += [z-l+beta >= 0]
constraints += [z >= 0]
constraints += [sum(x) == 1]
constraints += [x_pos <= K*b]
constraints += [x_pos >= 0]
constraints += [x_neg <= (1-b)*K]
constraints += [x_neg >= 0]
constraints += [x == x_pos - x_neg]
constraints += [M*sum(x_neg) - sum(x_pos) <= 0]

obj = mu_est*x
objective = cvx.Maximize(obj);

result = np.zeros(4)
print('Gamma    Asset 1    Asset 2    Asset 3    Asset 4')
for i in range(4):
    constraints += [(beta + 0.5*sum(z)) <= gamma[i]]
    prob = cvx.Problem(objective, constraints)
    
    prob.solve(solver = cvx.GUROBI)
    if (prob.status == "optimal"):
        # print('optimal value = ', prob.value)
        x0 = x[0].value;
        x1 = x[1].value;
        x2 = x[2].value;
        x3 = x[3].value;
        result[i] = prob.value;
        print(gamma[i], '  ', '%.5f  ' % x0, '%.5f ' % x1, ' ', '%.5f  ' % x2, '%.5f' % x3)
        #print(gamma[i],  x0,  x1,  x2,  x3)

Gamma    Asset 1    Asset 2    Asset 3    Asset 4
Using license file /Users/joehigh/gurobi.lic
0.08    -1.65957   0.00000    5.00000   -2.34043
0.06    -1.44681   0.00000    5.00000   -2.55319
0.04    -1.23404   0.00000    5.00000   -2.76596
0.02    -1.02128   0.00000    5.00000   -2.97872


In [ ]:
# Plot the mean-CVaR frontier

plt.title('Mean - CVaR Frontier')
plt.xlabel('CVaR')
plt.ylabel('Expected Return')
plt.plot(gamma, result, linewidth=2)
plt.grid(True)
plt.autoscale(enable=True, tight=True)
plt.axis([0.02,0.08,0.22,0.29])
plt.show()